## 1. A gentle example of using ReinforcementLearning.jl

In [ ]:
import Pkg;
Pkg.add("ReinforcementLearning");
using ReinforcementLearning;

In [ ]:
env = RandomWalk1D()

### random policy

In [ ]:
while true
    env(rand(A))
    is_terminated(env) && break
end

In [ ]:
run(
    RandomPolicy(),
    RandomWalk1D(),
    StopAfterEpisode(10),
    TotalRewardPerEpisode())

### tabular policy

In [ ]:
# create a tabular policy
S = state_space(env);
A = action_space(env);
NS, NA = length(S),A;
tabular_policy = TabularPolicy(;table=Dict(zip(1:NS, fill(2,NS))));

In [ ]:
run(
   tabular_policy,
   RandomWalk1D(),
   StopAfterEpisode(10),
   TotalRewardPerEpisode()
)

### `QBasedPolicy`

In [ ]:
Pkg.add("Flux")
using Flux: InvDecay

In [ ]:
# `MonteCarloLearner + EpsilonGreedyExplorer`
policy = QBasedPolicy(
   learner = MonteCarloLearner(;
           approximator=TabularQApproximator(
               ;n_state = NS,
               n_action = NA,
               opt = InvDecay(1.0)
           )
       ),
   explorer = EpsilonGreedyExplorer(0.1)
)

In [ ]:
run(
   policy,
   RandomWalk1D(),
   StopAfterEpisode(10),
   TotalRewardPerEpisode()
)

### wrap the policy + trajectory into the 'agent'

In [ ]:
agent = Agent(policy=policy, trajectory=VectorSARTTrajectory())

In [ ]:
run(agent, env, StopAfterEpisode(10), TotalRewardPerEpisode())